In [ ]:
from src.SensitivityAnalysis import load_data
import matplotlib.pyplot as plt
import pandas as pd
from src.SugarScape import SugarScape
from mesa import batch_run
import numpy as np
from tqdm import tqdm

from scipy.stats import kruskal, mannwhitneyu, shapiro
from itertools import combinations
import seaborn as sns
from itertools import product

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Finding Parameters

In [ ]:
data = load_data()
averaged = []
for i in range(len(data) // 10 - 1):
    trader_mean = data['Trader Count'].iloc[i*10:(i+1)*10].mean()
    gini_mean = data['Gini'].iloc[i*10:(i+1)*10].mean()
    
    if trader_mean > 0:
        averaged.append(1 / gini_mean * trader_mean)


In [ ]:
# Find where gini maximized 
worst = dict(data.iloc[np.argmin(averaged)])
best = dict(data.iloc[np.argmax(averaged)])

worst = {key: float(value) for (key,value) in worst.items()}
best = {key: float(value) for (key,value) in best.items()}

print(f'Worst case: {worst}')
print(f'Best case: {best}')

# Find closest to average
dist = (averaged - np.mean(averaged))**2
idx = np.argmin(dist)

typical = dict(data.iloc[idx * 10])
typical = {key: float(value) for (key,value) in typical.items()}
print(f'Typical: {typical}')


# Run Base Simulations

In [ ]:
## USe these map schemes based on your name
# Priyank: top-heavy
# Amish: uniform
# Ilia: split

map_scheme = 'split'

scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'map_scheme': map_scheme},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'map_scheme': map_scheme},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'map_scheme': map_scheme},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'map_scheme': map_scheme},                    # Fast evolving
]

results = []
replicates = 30
max_steps = 200

In [ ]:
for scenario in scenarios:
    batch = batch_run(SugarScape, scenario, number_processes=None,
                      iterations=replicates, max_steps=max_steps, display_progress=True, data_collection_period=1)
    for i in range(replicates):
        scenario["Gini"] = [val['Gini'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario["Trader Count"] = [val['Trader Count'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]

        results.append(list(scenario.values()))

# Convert to df and save
names = list(scenario.keys()) 
df = pd.DataFrame(results, columns=names)
df.to_csv(f'Experiments/Base/results_{map_scheme}.csv')

In [ ]:
def safe_eval(value):
    try:
        return eval(value)
    except:
        return value

names = ['Worst', 'Best', 'Typical', 'Balanced', 'Slow evolving', 'Fast evolving']
for map_scheme in ['uniform', 'top_heavy', 'split']:
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    df = pd.read_csv(f'Experiments/Base/results_{map_scheme}.csv')
    df['Gini'] = df['Gini'].apply(safe_eval)
    df['Trader Count'] = df['Trader Count'].apply(safe_eval)
    
    for i in range(len(scenarios)):
        # Get gini and trader count of every scenario
        gini = df.iloc[i*replicates:(i+1)*replicates]['Gini']
        trader = df.iloc[i*replicates:(i+1)*replicates]['Trader Count']
        
        # Convert to 2d np arrays
        gini = np.vstack(gini)
        trader = np.vstack(trader)
        
        # Get means and ci
        gini_mean = np.mean(gini, axis=0)
        gini_ci = np.std(gini, ddof=1, axis=0) / replicates * 1.96
        trader_mean = np.mean(trader, axis=0)
        trader_ci = np.std(trader, ddof=1, axis=0) / replicates * 1.96
        
        # Plotting
        x = np.arange(len(gini_mean))
        ax[0].plot(x, gini_mean, label=names[i])
        ax[0].fill_between(x, gini_mean + gini_ci, gini_mean - gini_ci, alpha=0.5)
        ax[1].plot(x, trader_mean, label=names[i])
        ax[1].fill_between(x, trader_mean + trader_ci, trader_mean - trader_ci, alpha=0.5)
    
    # Set labels, grid and title
    ax[1].legend(bbox_to_anchor=(1, 1.05))
    ax[0].set_title('Gini')
    ax[1].set_title('Trader Count')
    for i in range(2):
        ax[i].grid()
        ax[i].set_xlabel('Timesteps')
    ax[0].set_ylabel('Gini Coefficient')
    ax[1].set_ylabel('Number of Traders')
    
    # Save figure
    fig.suptitle(f'{map_scheme}')
    fig.tight_layout()
    fig.savefig(f'Experiments/Base/plot_{map_scheme}.png', dpi=300, bbox_inches='tight')

# Running Experiments

In [ ]:
## Use these map schemes based on your name
# Priyank: top-heavy
# Amish: uniform
# Ilia: split

map_scheme = 'uniform'

scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'map_scheme': map_scheme},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'map_scheme': map_scheme},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'map_scheme': map_scheme},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'map_scheme': map_scheme},                    # Fast evolving
]


tax_systems = [
    ("progressive", "needs"),
    ("flat", "flat"),
    ("regressive", "random"),
    ("luxury", "progressive"),
    ("progressive", "progressive")
]

tax_rates = [0.1, 0.25, 0.4]

results = []
replicates = 30
max_steps = 200

In [ ]:
with tqdm(total=len(scenarios), ncols=70) as pbar:
    for scenario in scenarios:
        for tax_system in tax_systems:
            for tax_rate in tax_rates:
                scenario_copy = scenario.copy()
                # Adding taxsystem and rate into scenario
                scenario_copy['tax_scheme'] = tax_system[0]
                scenario_copy['distributer_scheme'] = tax_system[1]
                scenario_copy['tax_rate'] = tax_rate
        
                batch = batch_run(SugarScape, scenario_copy, number_processes=None,
                                  iterations=replicates, max_steps=max_steps, display_progress=False, data_collection_period=1)
                for i in range(replicates):
                    scenario_copy["Gini"] = [val['Gini'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy["Trader Count"] = [val['Trader Count'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
    
                results.append(list(scenario_copy.values()))
    
        pbar.update(1)

# Convert to df and save
names = list(scenario_copy.keys()) 
df = pd.DataFrame(results, columns=names)
df.to_csv(f'Experiments/results_{map_scheme}.csv')

# Updated Final Experiment

In [ ]:
import logging
from multiprocessing import Pool, cpu_count
# from src.SugarScape import SugarScape, batch_run

def setup_logger():
    """Set up a logger for simulation."""
    logger = logging.getLogger("simulation")
    handler = logging.FileHandler("simulation.log")
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger

map_scheme = 'top_heavy'

scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'map_scheme': map_scheme},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'map_scheme': map_scheme},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'map_scheme': map_scheme},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'map_scheme': map_scheme},                    # Fast evolving
]

tax_systems = [
    ("progressive", "needs"),
    ("flat", "flat"),
    ("regressive", "random"),
    ("luxury", "progressive"),
    ("progressive", "progressive")
]

tax_rates = [0.1, 0.25, 0.4]

results = []
replicates = 10
max_steps = 200

def run_experiment(scenario):
    """Run the SugarScape model with given parameters and return results."""
    logger = setup_logger()
    results = []
    for tax_system in tax_systems:
        for tax_rate in tax_rates:
            scenario_copy = scenario.copy()
            scenario_copy['tax_scheme'] = tax_system[0]
            scenario_copy['distributer_scheme'] = tax_system[1]
            scenario_copy['tax_rate'] = tax_rate
            
            logger.info(f"Running experiment: {scenario_copy}")
            batch = batch_run(SugarScape, scenario_copy, number_processes=None,
                              iterations=replicates, max_steps=max_steps, display_progress=False, data_collection_period=1)
            for i in range(replicates):
                scenario_result = scenario_copy.copy()
                scenario_result["Gini"] = [val['Gini'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                scenario_result["Trader Count"] = [val['Trader Count'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                results.append(list(scenario_result.values()))
    return results

with tqdm(total=len(scenarios), ncols=70) as pbar:
    for scenario in scenarios:
        results.extend(run_experiment(scenario))
        pbar.update(1)

# Convert to df and save
names = list(scenario.keys()) + ['tax_scheme', 'distributer_scheme', 'tax_rate', 'Gini', 'Trader Count']
df = pd.DataFrame(results, columns=names)
df.to_csv(f'Experiments/results_v4_{map_scheme}.csv', index=False)


# Visualisation of Time Series Plots

In [ ]:
# Function to load and process the data
def load_and_process(file_path):
    """Load CSV file and process the data."""
    data = pd.read_csv(file_path)
    
    def convert_str_to_list(list_str):
        # Strip unnecessary characters and split the string
        list_str = list_str.replace('np.float64(', '').replace(')', '').strip('[]')
        list_str = list_str.split(',')
        return [float(x.strip()) for x in list_str]

    # Apply the conversion to the relevant columns
    data['Gini'] = data['Gini'].apply(lambda x: convert_str_to_list(x) if isinstance(x, str) else x)
    data['Trader Count'] = data['Trader Count'].apply(lambda x: convert_str_to_list(x) if isinstance(x, str) else x)
    
    return data

# Load and process data
uniform_data = load_and_process('Experiments/results_v4_uniform.csv')
top_heavy_data = load_and_process('Experiments/results_v4_top_heavy.csv')
split_data = load_and_process('Experiments/results_v4_split.csv')

# Combine data into one DataFrame
data = pd.concat([uniform_data, top_heavy_data, split_data])

# Define scenarios
scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'Scenario': 'Worst case'},
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'Scenario': 'Best case'},
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'Scenario': 'Average case'},
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'Scenario': 'Balanced case'},
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'Scenario': 'Slow evolving'},
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'Scenario': 'Fast evolving'}
]

# Assign scenarios to data
data['Scenario'] = 'Other'
for scenario in scenarios:
    data.loc[
        (data['vision_mean'] == scenario['vision_mean']) &
        (data['metabolism_mean'] == scenario['metabolism_mean']) &
        (data['max_age_mean'] == scenario['max_age_mean']) &
        (data['repopulate_factor'] == scenario['repopulate_factor']) &
        (data['cell_regeneration'] == scenario['cell_regeneration']),
        'Scenario'] = scenario['Scenario']

# Function to plot metrics with 95% CI
def plot_metric_with_ci(data, metric, y_label, scenario_name, map_scheme, tax_rate):
    fig, axs = plt.subplots(1, 2, figsize=(18, 8), constrained_layout=True)
    
    for ax, metric, y_label in zip(axs, ['Gini', 'Trader Count'], ['Gini Coefficient', 'Trader Count']):
        subset = data[(data['Scenario'] == scenario_name) & (data['map_scheme'] == map_scheme) & (data['tax_rate'] == tax_rate)]
        for (tax_scheme, distributer_scheme), group_data in subset.groupby(['tax_scheme', 'distributer_scheme']):
            values = np.array(group_data[metric].tolist())
            mean_values = np.mean(values, axis=0)
            std_error = np.std(values, axis=0) / np.sqrt(values.shape[0])
            ci95 = 1.96 * std_error

            time_steps = np.arange(values.shape[1])
            label = f"{tax_scheme}, {distributer_scheme}"
            ax.plot(time_steps, mean_values, label=label)
            ax.fill_between(time_steps, mean_values - ci95, mean_values + ci95, alpha=0.3)
        
        ax.set_title(f'{y_label} - {map_scheme} - Tax Rate {tax_rate}')
        ax.set_xlabel('Time Steps')
        ax.set_ylabel(y_label)
        ax.legend(loc='best')

    fig.suptitle(f'{scenario_name} - {map_scheme} - Tax Rate {tax_rate}', fontsize=16)
    plt.show()

# Plot results with 95% CI for Gini and Trader Count
for map_scheme in ['top_heavy', 'split', 'uniform']:
    for tax_rate in [0.1, 0.25, 0.4]:
        for scenario in data['Scenario'].unique():
            plot_metric_with_ci(data, 'Gini', 'Gini Coefficient', scenario, map_scheme, tax_rate)


# Statistical Testing

In [ ]:
# Function to load and process the data
def load_and_process(file_path):
    """Load CSV file and process the data."""
    data = pd.read_csv(file_path)
    
    def convert_str_to_list(list_str):
        # Strip unnecessary characters and split the string
        list_str = list_str.replace('np.float64(', '').replace(')', '').strip('[]')
        list_str = list_str.split(',')
        return [float(x.strip()) for x in list_str]

    # Apply the conversion to the relevant columns
    data['Gini'] = data['Gini'].apply(lambda x: convert_str_to_list(x) if isinstance(x, str) else x)
    data['Trader Count'] = data['Trader Count'].apply(lambda x: convert_str_to_list(x) if isinstance(x, str) else x)
    
    return data

# Load and process data
uniform_data = load_and_process('Experiments/results_v4_uniform.csv')
top_heavy_data = load_and_process('Experiments/results_v4_top_heavy.csv')
split_data = load_and_process('Experiments/results_v4_split.csv')

# Combine data into one DataFrame
data = pd.concat([uniform_data, top_heavy_data, split_data])

# Define scenarios
scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'Scenario': 'Worst case'},
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'Scenario': 'Best case'},
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'Scenario': 'Average case'},
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'Scenario': 'Balanced case'},
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'Scenario': 'Slow evolving'},
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'Scenario': 'Fast evolving'}
]

# Assign scenarios to data
data['Scenario'] = 'Other'
for scenario in scenarios:
    data.loc[
        (data['vision_mean'] == scenario['vision_mean']) &
        (data['metabolism_mean'] == scenario['metabolism_mean']) &
        (data['max_age_mean'] == scenario['max_age_mean']) &
        (data['repopulate_factor'] == scenario['repopulate_factor']) &
        (data['cell_regeneration'] == scenario['cell_regeneration']),
        'Scenario'] = scenario['Scenario']

# Function to check normality using the Shapiro-Wilk test
def check_normality(data, column, scenarios):
    normality_results = {}
    for scenario in scenarios:
        scenario_name = scenario['Scenario']
        subset = data[data['Scenario'] == scenario_name]
        flattened_data = np.concatenate(subset[column].values)
        if len(flattened_data) > 5000:
            flattened_data = flattened_data[:5000]  # Shapiro-Wilk test has a limit on sample size
        stat, p_value = shapiro(flattened_data)
        normality_results[scenario_name] = p_value
        if p_value < 0.05:
            print(f"The {column} data for {scenario_name} is not normally distributed (p-value = {p_value}).")
        else:
            print(f"The {column} data for {scenario_name} is normally distributed (p-value = {p_value}).")
    return normality_results

# Check normality for Gini and Trader Count
gini_normality = check_normality(data, 'Gini', scenarios)
trader_count_normality = check_normality(data, 'Trader Count', scenarios)


In [ ]:

# Function to plot heatmaps for post-hoc comparisons
def plot_posthoc_heatmap(posthoc_results, map_scheme, scenario_name, metric):
    if not posthoc_results or scenario_name not in posthoc_results:
        print(f"No significant differences found for {metric} in {scenario_name} for {map_scheme} map")
        return

    posthoc_result = posthoc_results[scenario_name]
    unique_groups = list(set([pair[0] for pair in posthoc_result.keys()] + [pair[1] for pair in posthoc_result.keys()]))
    
    # Create a MultiIndex for handling tuples as index and columns
    multi_index = pd.MultiIndex.from_tuples(unique_groups, names=["tax_scheme", "distributer_scheme", "tax_rate"])
    
    # Initialize DataFrame for heatmap data with NaNs
    heatmap_data = pd.DataFrame(index=multi_index, columns=multi_index, dtype=float)
    
    # Fill DataFrame with p-values
    for pair, p_value in posthoc_result.items():
        heatmap_data.loc[pair[0], pair[1]] = p_value
        heatmap_data.loc[pair[1], pair[0]] = p_value

    # Fill diagonal with zeros as comparisons of the same group should be zero
    for group in unique_groups:
        heatmap_data.loc[group, group] = 0
    
    plt.figure(figsize=(12, 10))
    ax = sns.heatmap(heatmap_data, cmap='coolwarm', cbar=True, center=0.05, linewidths=0.5, linecolor='black')
    plt.title(f'Post-hoc Comparisons for {metric} - {scenario_name} ({map_scheme} Map)')
    plt.xlabel('Tax System Combinations')
    plt.ylabel('Tax System Combinations')
    
    # Add a legend for significance levels
    cbar = ax.collections[0].colorbar
    cbar.set_label('p-value')
    
    # Add a red dotted line at the significance threshold
    cbar.ax.axhline(y=0.05, color='red', linestyle='--', linewidth=2)
    cbar.ax.text(1.2, 0.05, 'Significance Threshold (0.05)', color='red', ha='center', va='center', rotation=90)
    
    plt.show()

# Function to convert list columns to mean values
def convert_list_to_mean(df, column):
    return df[column].apply(lambda x: np.mean(x) if isinstance(x, list) else x)

data['Gini_mean'] = convert_list_to_mean(data, 'Gini')
data['Trader Count_mean'] = convert_list_to_mean(data, 'Trader Count')

# Function to perform Kruskal-Wallis test
def kruskal_test(data, column, scenarios):
    results = {}
    for scenario in scenarios:
        scenario_name = scenario['Scenario']
        subset = data[data['Scenario'] == scenario_name]
        values = [np.mean(x) for x in subset[column].values]
        groups = subset[['tax_scheme', 'distributer_scheme', 'tax_rate']].apply(tuple, axis=1).values
        
        # Perform Kruskal-Wallis test
        groups_unique = np.unique(groups)
        data_grouped = [np.array([values[i] for i in range(len(values)) if groups[i] == group]) for group in groups_unique]
        
        stat, p_value = kruskal(*data_grouped)
        results[scenario_name] = (stat, p_value)
        
    return results

# Function to perform one-sided post-hoc pairwise comparisons using Mann-Whitney U test
def posthoc_comparisons(data, column, scenarios, alternative='two-sided'):
    posthoc_results = {}
    for scenario in scenarios:
        scenario_name = scenario['Scenario']
        subset = data[data['Scenario'] == scenario_name]
        values = [np.mean(x) for x in subset[column].values]
        groups = subset[['tax_scheme', 'distributer_scheme', 'tax_rate']].apply(tuple, axis=1).values
        
        # Perform Mann-Whitney U test for pairwise comparisons
        unique_groups = np.unique(groups)
        group_pairs = list(product(unique_groups, repeat=2))
        p_values = {}
        
        for pair in group_pairs:
            if pair[0] == pair[1]:
                continue
            group1 = [values[i] for i in range(len(values)) if groups[i] == pair[0]]
            group2 = [values[i] for i in range(len(values)) if groups[i] == pair[1]]
            stat, p_value = mannwhitneyu(group1, group2, alternative=alternative)
            p_values[pair] = p_value
        
        posthoc_results[scenario_name] = p_values
    
    return posthoc_results

# Function to rank combinations
def rank_combinations(data, map_scheme, scenario_name, metric):
    filtered_data = data[(data['map_scheme'] == map_scheme) & (data['Scenario'] == scenario_name)]
    agg_data = filtered_data.groupby(['tax_scheme', 'distributer_scheme', 'tax_rate']).agg({
        f'{metric}_mean': 'mean'
    }).reset_index()
    agg_data['Rank'] = agg_data[f'{metric}_mean'].rank(method='min', ascending=(metric == 'Gini'))
    return agg_data

# Example usage for visualizations
# Define map schemes
map_schemes = ['uniform', 'top_heavy', 'split']

# Iterate through all map schemes and scenarios
for map_scheme, scenario in product(map_schemes, scenarios):
    scenario_name = scenario['Scenario']
    map_data = data[data['map_scheme'] == map_scheme]  # Assuming there's a column 'map_scheme' to filter data
    
    # Filter posthoc results for the specific map scheme
    gini_posthoc_map = posthoc_comparisons(map_data, 'Gini', scenarios, alternative='less') if any(p < 0.05 for _, p in kruskal_test(map_data, 'Gini', scenarios).values()) else None
    trader_count_posthoc_map = posthoc_comparisons(map_data, 'Trader Count', scenarios, alternative='greater') if any(p < 0.05 for _, p in kruskal_test(map_data, 'Trader Count', scenarios).values()) else None
    
    plot_posthoc_heatmap(gini_posthoc_map, map_scheme, scenario_name, 'Gini')
    plot_posthoc_heatmap(trader_count_posthoc_map, map_scheme, scenario_name, 'Trader Count')
    
    # Rank combinations
    gini_ranks = rank_combinations(data, map_scheme, scenario_name, 'Gini')
    trader_count_ranks = rank_combinations(data, map_scheme, scenario_name, 'Trader Count')
    
    # Print best combinations
    best_gini_combination = gini_ranks.loc[gini_ranks['Rank'].idxmin()]
    best_trader_combination = trader_count_ranks.loc[trader_count_ranks['Rank'].idxmin()]
    
    print(f'Best Gini combination for {map_scheme} map and {scenario_name} scenario:')
    print(best_gini_combination)
    
    print(f'Best Trader Count combination for {map_scheme} map and {scenario_name} scenario:')
    print(best_trader_combination)
